In [1]:
!pip install --upgrade pip

    100% |████████████████████████████████| 1.4MB 54.3MB/s ta 0:00:01
  Found existing installation: pip 9.0.1
    Uninstalling pip-9.0.1:
      Successfully uninstalled pip-9.0.1


In [2]:
!pip install --upgrade Pillow

Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
     |████████████████████████████████| 2.1MB 45.4MB/s eta 0:00:01�█                            | 256kB 45.4MB/s eta 0:00:01
  Found existing installation: Pillow 5.0.0
    Uninstalling Pillow-5.0.0:
      Successfully uninstalled Pillow-5.0.0


In [ ]:
!pip install --upgrade scikit-image==0.15

Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
     |████████████████████████████████| 26.3MB 53.5MB/s eta 0:00:01    |▊                               | 552kB 53.5MB/s eta 0:00:01[K     |██████                          | 5.0MB 53.5MB/s eta 0:00:01        | 10.7MB 53.5MB/s eta 0:00:01    |██████████████████▉             | 15.5MB 53.5MB/s eta 0:00:01        | 19.4MB 53.5MB/s eta 0:00:01��████████████████████▊  | 24.5MB 53.5MB/s eta 0:00:01
  Found existing installation: scikit-image 0.13.0
    Uninstalling scikit-image-0.13.0:
      Successfully uninstalled scikit-image-0.13.0


In [ ]:
!pip install -r requirements.txt

Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
     |██████████████████████�███▊     | 352.9MB 98.7MB/s eta 0:00:01                          | 3.8MB 78.9MB/s eta 0:00:06MB/s eta 0:00:06               | 13.4MB 78.9MB/s eta 0:00:06                     | 18.5MB 78.9MB/s eta 0:00:06.9MB 78.9MB/s eta 0:00:06:00:0505| 43.2MB 78.9MB/s eta 0:00:05                         | 48.4MB 78.9MB/s eta 0:00:05| 52.8MB 78.9MB/s eta 0:00:05                     | 57.7MB 78.9MB/s eta 0:00:058.9MB/s eta 0:00:05 |█████▏                          | 67.7MB 78.9MB/s eta 0:00:05 |█████▌                          | 72.4MB 78.9MB/s eta 0:00:05�▉                          | 76.8MB 78.9MB/s eta 0:00:05           | 82.0MB 78.9MB/s eta 0:00:05    |██████▌                         | 86.0MB 78.9MB/s eta 0:00:05           | 91.1MB 98.1MB/s eta 0:00:044:00:046MB 98.1MB/s eta 0:00:04��████▎                       | 108.8MB 98.1MB/s eta 0:00:04��████▋                       | 113.6MB 98.1MB/s eta 0:00:04�

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
import os
import time
import random
import numpy as np
import scipy, multiprocessing
import tensorflow as tf
import tensorlayer as tl
from model import get_G, get_D
from config import config


In [ ]:
###====================== HYPER-PARAMETERS ===========================###

batch_size = 8  # use 8 if your GPU memory is small, and change [4, 4] in tl.vis.save_images to [2, 4]
lr_init = config.TRAIN.lr_init
beta1 = config.TRAIN.beta1
## initialize G
n_epoch_init = config.TRAIN.n_epoch_init

n_epoch = config.TRAIN.n_epoch
lr_decay = config.TRAIN.lr_decay
decay_every = config.TRAIN.decay_every
shuffle_buffer_size = 128


In [ ]:
# create folders to save result images and trained models
save_dir = "samples"
tl.files.exists_or_mkdir(save_dir)
checkpoint_dir = "checkpoint"
tl.files.exists_or_mkdir(checkpoint_dir)

In [ ]:
def get_train_data():
    # load dataset
    train_hr_img_list = sorted(tl.files.load_file_list(path=config.TRAIN.hr_img_path, regx='.*.png', printable=False))[0:200]

    ## If your machine have enough memory, please pre-load the entire train set.
    train_hr_imgs = tl.vis.read_images(train_hr_img_list, path=config.TRAIN.hr_img_path, n_threads=32)
        
    # dataset API and augmentation
    def generator_train():
        for img in train_hr_imgs:
            yield img
    def _map_fn_train(img):
        hr_patch = tf.image.random_crop(img, [384, 384, 3])
        hr_patch = hr_patch / (255. / 2.)
        hr_patch = hr_patch - 1.
        hr_patch = tf.image.random_flip_left_right(hr_patch)
        lr_patch = downscale_hr_patches(hr_patch)
        return lr_patch, hr_patch
    train_ds = tf.data.Dataset.from_generator(generator_train, output_types=(tf.float32))
    train_ds = train_ds.map(_map_fn_train, num_parallel_calls=multiprocessing.cpu_count())
        # train_ds = train_ds.repeat(n_epoch_init + n_epoch)
    train_ds = train_ds.shuffle(shuffle_buffer_size)
    train_ds = train_ds.prefetch(buffer_size=2)
    train_ds = train_ds.batch(batch_size)
        # value = train_ds.make_one_shot_iterator().get_next()
    return train_ds

def downscale_hr_patches(hr_patch):
    return tf.image.resize(hr_patch, size=[96, 96])

In [ ]:
VGG = tl.models.vgg19(pretrained=True, end_with='pool4', mode='static')
train_ds = get_train_data()

In [ ]:
G = get_G((batch_size, 96, 96, 3))
D = get_D((batch_size, 384, 384, 3))


lr_v = tf.Variable(lr_init)
g_optimizer_init = tf.optimizers.Adam(lr_v, beta_1=beta1)
g_optimizer = tf.optimizers.Adam(lr_v, beta_1=beta1)
d_optimizer = tf.optimizers.Adam(lr_v, beta_1=beta1)

G.train()
D.train()
VGG.train()

In [ ]:
from modelarts.session import Session
session = Session() 

In [23]:
## initialize learning (G)
n_step_epoch = round(n_epoch_init // batch_size)
for epoch in range(n_epoch_init):
    for step, (lr_patchs, hr_patchs) in enumerate(train_ds):
        if lr_patchs.shape[0] != batch_size: # if the remaining data in this epoch < batch_size
            break
        step_time = time.time()
        with tf.GradientTape() as tape:
            fake_hr_patchs = G(lr_patchs)

            mse_loss = tl.cost.mean_squared_error(fake_hr_patchs, hr_patchs, is_mean=True)

        grad = tape.gradient(mse_loss, G.trainable_weights)
        g_optimizer_init.apply_gradients(zip(grad, G.trainable_weights))
        print("Epoch: [{}/{}] step: [{}/{}] time: {:.3f}s, mse: {:.3f} ".format(
            epoch, n_epoch_init, step, n_step_epoch, time.time() - step_time, mse_loss))
    if (epoch != 0) and (epoch % 10 == 0):
        tl.vis.save_images(fake_hr_patchs.numpy(), [2, 4], os.path.join(save_dir, 'train_g_init_{}.png'.format(epoch)))
        

Epoch: [0/100] step: [0/12] time: 1.163s, mse: 0.306 
Epoch: [0/100] step: [1/12] time: 0.460s, mse: 0.433 
Epoch: [0/100] step: [2/12] time: 0.457s, mse: 0.252 
Epoch: [0/100] step: [3/12] time: 0.470s, mse: 0.367 
Epoch: [0/100] step: [4/12] time: 0.447s, mse: 0.264 
Epoch: [0/100] step: [5/12] time: 0.466s, mse: 0.257 
Epoch: [0/100] step: [6/12] time: 0.479s, mse: 0.313 
Epoch: [0/100] step: [7/12] time: 0.453s, mse: 0.329 
Epoch: [0/100] step: [8/12] time: 0.366s, mse: 0.197 
Epoch: [0/100] step: [9/12] time: 0.357s, mse: 0.310 
Epoch: [0/100] step: [10/12] time: 0.532s, mse: 0.395 
Epoch: [0/100] step: [11/12] time: 0.372s, mse: 0.299 
Epoch: [0/100] step: [12/12] time: 0.360s, mse: 0.312 
Epoch: [0/100] step: [13/12] time: 0.358s, mse: 0.307 
Epoch: [0/100] step: [14/12] time: 0.358s, mse: 0.270 
Epoch: [0/100] step: [15/12] time: 0.368s, mse: 0.331 
Epoch: [0/100] step: [16/12] time: 0.372s, mse: 0.295 
Epoch: [0/100] step: [17/12] time: 0.359s, mse: 0.231 
Epoch: [0/100] step:

Epoch: [6/100] step: [1/12] time: 0.465s, mse: 0.058 
Epoch: [6/100] step: [2/12] time: 0.445s, mse: 0.028 
Epoch: [6/100] step: [3/12] time: 0.449s, mse: 0.042 
Epoch: [6/100] step: [4/12] time: 0.447s, mse: 0.057 
Epoch: [6/100] step: [5/12] time: 0.467s, mse: 0.058 
Epoch: [6/100] step: [6/12] time: 0.463s, mse: 0.043 
Epoch: [6/100] step: [7/12] time: 0.638s, mse: 0.040 
Epoch: [6/100] step: [8/12] time: 0.362s, mse: 0.038 
Epoch: [6/100] step: [9/12] time: 0.356s, mse: 0.065 
Epoch: [6/100] step: [10/12] time: 0.368s, mse: 0.043 
Epoch: [6/100] step: [11/12] time: 0.356s, mse: 0.042 
Epoch: [6/100] step: [12/12] time: 0.358s, mse: 0.035 
Epoch: [6/100] step: [13/12] time: 0.358s, mse: 0.026 
Epoch: [6/100] step: [14/12] time: 0.356s, mse: 0.057 
Epoch: [6/100] step: [15/12] time: 0.365s, mse: 0.067 
Epoch: [6/100] step: [16/12] time: 0.358s, mse: 0.058 
Epoch: [6/100] step: [17/12] time: 0.356s, mse: 0.054 
Epoch: [6/100] step: [18/12] time: 0.494s, mse: 0.065 
Epoch: [6/100] step

Epoch: [12/100] step: [1/12] time: 0.467s, mse: 0.042 
Epoch: [12/100] step: [2/12] time: 0.417s, mse: 0.017 
Epoch: [12/100] step: [3/12] time: 0.442s, mse: 0.030 
Epoch: [12/100] step: [4/12] time: 0.462s, mse: 0.037 
Epoch: [12/100] step: [5/12] time: 0.571s, mse: 0.036 
Epoch: [12/100] step: [6/12] time: 0.446s, mse: 0.029 
Epoch: [12/100] step: [7/12] time: 0.392s, mse: 0.038 
Epoch: [12/100] step: [8/12] time: 0.355s, mse: 0.031 
Epoch: [12/100] step: [9/12] time: 0.359s, mse: 0.051 
Epoch: [12/100] step: [10/12] time: 0.366s, mse: 0.041 
Epoch: [12/100] step: [11/12] time: 0.352s, mse: 0.034 
Epoch: [12/100] step: [12/12] time: 0.354s, mse: 0.044 
Epoch: [12/100] step: [13/12] time: 0.352s, mse: 0.023 
Epoch: [12/100] step: [14/12] time: 0.355s, mse: 0.045 
Epoch: [12/100] step: [15/12] time: 0.468s, mse: 0.039 
Epoch: [12/100] step: [16/12] time: 0.357s, mse: 0.019 
Epoch: [12/100] step: [17/12] time: 0.357s, mse: 0.039 
Epoch: [12/100] step: [18/12] time: 0.364s, mse: 0.042 
E

Epoch: [17/100] step: [24/12] time: 0.372s, mse: 0.034 
Epoch: [18/100] step: [0/12] time: 0.448s, mse: 0.043 
Epoch: [18/100] step: [1/12] time: 0.466s, mse: 0.039 
Epoch: [18/100] step: [2/12] time: 0.468s, mse: 0.021 
Epoch: [18/100] step: [3/12] time: 0.412s, mse: 0.030 
Epoch: [18/100] step: [4/12] time: 0.579s, mse: 0.039 
Epoch: [18/100] step: [5/12] time: 0.406s, mse: 0.034 
Epoch: [18/100] step: [6/12] time: 0.463s, mse: 0.032 
Epoch: [18/100] step: [7/12] time: 0.420s, mse: 0.029 
Epoch: [18/100] step: [8/12] time: 0.353s, mse: 0.025 
Epoch: [18/100] step: [9/12] time: 0.350s, mse: 0.043 
Epoch: [18/100] step: [10/12] time: 0.354s, mse: 0.039 
Epoch: [18/100] step: [11/12] time: 0.360s, mse: 0.040 
Epoch: [18/100] step: [12/12] time: 0.350s, mse: 0.057 
Epoch: [18/100] step: [13/12] time: 0.506s, mse: 0.031 
Epoch: [18/100] step: [14/12] time: 0.354s, mse: 0.042 
Epoch: [18/100] step: [15/12] time: 0.359s, mse: 0.051 
Epoch: [18/100] step: [16/12] time: 0.352s, mse: 0.023 
Ep

Epoch: [23/100] step: [22/12] time: 0.366s, mse: 0.055 
Epoch: [23/100] step: [23/12] time: 0.364s, mse: 0.024 
Epoch: [23/100] step: [24/12] time: 0.359s, mse: 0.018 
Epoch: [24/100] step: [0/12] time: 0.455s, mse: 0.033 
Epoch: [24/100] step: [1/12] time: 0.469s, mse: 0.040 
Epoch: [24/100] step: [2/12] time: 0.629s, mse: 0.022 
Epoch: [24/100] step: [3/12] time: 0.416s, mse: 0.014 
Epoch: [24/100] step: [4/12] time: 0.468s, mse: 0.034 
Epoch: [24/100] step: [5/12] time: 0.446s, mse: 0.025 
Epoch: [24/100] step: [6/12] time: 0.460s, mse: 0.028 
Epoch: [24/100] step: [7/12] time: 0.421s, mse: 0.029 
Epoch: [24/100] step: [8/12] time: 0.363s, mse: 0.030 
Epoch: [24/100] step: [9/12] time: 0.381s, mse: 0.040 
Epoch: [24/100] step: [10/12] time: 0.356s, mse: 0.036 
Epoch: [24/100] step: [11/12] time: 0.357s, mse: 0.028 
Epoch: [24/100] step: [12/12] time: 0.527s, mse: 0.023 
Epoch: [24/100] step: [13/12] time: 0.359s, mse: 0.019 
Epoch: [24/100] step: [14/12] time: 0.365s, mse: 0.053 
Ep

...

Epoch: [71/100] step: [6/12] time: 0.428s, mse: 0.018 
Epoch: [71/100] step: [7/12] time: 0.422s, mse: 0.020 
Epoch: [71/100] step: [8/12] time: 0.358s, mse: 0.021 
Epoch: [71/100] step: [9/12] time: 0.353s, mse: 0.031 
Epoch: [71/100] step: [10/12] time: 0.351s, mse: 0.027 
Epoch: [71/100] step: [11/12] time: 0.351s, mse: 0.016 
Epoch: [71/100] step: [12/12] time: 0.509s, mse: 0.022 
Epoch: [71/100] step: [13/12] time: 0.355s, mse: 0.013 
Epoch: [71/100] step: [14/12] time: 0.358s, mse: 0.028 
Epoch: [71/100] step: [15/12] time: 0.352s, mse: 0.028 
Epoch: [71/100] step: [16/12] time: 0.349s, mse: 0.022 
Epoch: [71/100] step: [17/12] time: 0.353s, mse: 0.030 
Epoch: [71/100] step: [18/12] time: 0.361s, mse: 0.026 
Epoch: [71/100] step: [19/12] time: 0.355s, mse: 0.011 
Epoch: [71/100] step: [20/12] time: 0.357s, mse: 0.015 
Epoch: [71/100] step: [21/12] time: 0.356s, mse: 0.018 
Epoch: [71/100] step: [22/12] time: 0.538s, mse: 0.050 
Epoch: [71/100] step: [23/12] time: 0.354s, mse: 0.0

Epoch: [77/100] step: [4/12] time: 0.455s, mse: 0.028 
Epoch: [77/100] step: [5/12] time: 0.433s, mse: 0.019 
Epoch: [77/100] step: [6/12] time: 0.438s, mse: 0.020 
Epoch: [77/100] step: [7/12] time: 0.434s, mse: 0.021 
Epoch: [77/100] step: [8/12] time: 0.360s, mse: 0.015 
Epoch: [77/100] step: [9/12] time: 0.363s, mse: 0.020 
Epoch: [77/100] step: [10/12] time: 0.489s, mse: 0.020 
Epoch: [77/100] step: [11/12] time: 0.353s, mse: 0.015 
Epoch: [77/100] step: [12/12] time: 0.356s, mse: 0.029 
Epoch: [77/100] step: [13/12] time: 0.354s, mse: 0.009 
Epoch: [77/100] step: [14/12] time: 0.352s, mse: 0.023 
Epoch: [77/100] step: [15/12] time: 0.357s, mse: 0.018 
Epoch: [77/100] step: [16/12] time: 0.363s, mse: 0.016 
Epoch: [77/100] step: [17/12] time: 0.362s, mse: 0.024 
Epoch: [77/100] step: [18/12] time: 0.363s, mse: 0.026 
Epoch: [77/100] step: [19/12] time: 0.351s, mse: 0.014 
Epoch: [77/100] step: [20/12] time: 0.520s, mse: 0.014 
Epoch: [77/100] step: [21/12] time: 0.353s, mse: 0.015

Epoch: [83/100] step: [2/12] time: 0.477s, mse: 0.019 
Epoch: [83/100] step: [3/12] time: 0.449s, mse: 0.019 
Epoch: [83/100] step: [4/12] time: 0.444s, mse: 0.023 
Epoch: [83/100] step: [5/12] time: 0.469s, mse: 0.016 
Epoch: [83/100] step: [6/12] time: 0.473s, mse: 0.021 
Epoch: [83/100] step: [7/12] time: 0.607s, mse: 0.021 
Epoch: [83/100] step: [8/12] time: 0.352s, mse: 0.020 
Epoch: [83/100] step: [9/12] time: 0.371s, mse: 0.034 
Epoch: [83/100] step: [10/12] time: 0.358s, mse: 0.022 
Epoch: [83/100] step: [11/12] time: 0.357s, mse: 0.012 
Epoch: [83/100] step: [12/12] time: 0.360s, mse: 0.021 
Epoch: [83/100] step: [13/12] time: 0.356s, mse: 0.019 
Epoch: [83/100] step: [14/12] time: 0.351s, mse: 0.020 
Epoch: [83/100] step: [15/12] time: 0.353s, mse: 0.033 
Epoch: [83/100] step: [16/12] time: 0.360s, mse: 0.016 
Epoch: [83/100] step: [17/12] time: 0.362s, mse: 0.031 
Epoch: [83/100] step: [18/12] time: 0.528s, mse: 0.028 
Epoch: [83/100] step: [19/12] time: 0.358s, mse: 0.014 


Epoch: [89/100] step: [0/12] time: 0.497s, mse: 0.019 
Epoch: [89/100] step: [1/12] time: 0.469s, mse: 0.029 
Epoch: [89/100] step: [2/12] time: 0.462s, mse: 0.018 
Epoch: [89/100] step: [3/12] time: 0.415s, mse: 0.029 
Epoch: [89/100] step: [4/12] time: 0.463s, mse: 0.030 
Epoch: [89/100] step: [5/12] time: 0.451s, mse: 0.015 
Epoch: [89/100] step: [6/12] time: 0.586s, mse: 0.016 
Epoch: [89/100] step: [7/12] time: 0.453s, mse: 0.023 
Epoch: [89/100] step: [8/12] time: 0.361s, mse: 0.017 
Epoch: [89/100] step: [9/12] time: 0.388s, mse: 0.022 
Epoch: [89/100] step: [10/12] time: 0.376s, mse: 0.024 
Epoch: [89/100] step: [11/12] time: 0.354s, mse: 0.010 
Epoch: [89/100] step: [12/12] time: 0.364s, mse: 0.020 
Epoch: [89/100] step: [13/12] time: 0.361s, mse: 0.022 
Epoch: [89/100] step: [14/12] time: 0.361s, mse: 0.025 
Epoch: [89/100] step: [15/12] time: 0.574s, mse: 0.011 
Epoch: [89/100] step: [16/12] time: 0.364s, mse: 0.025 
Epoch: [89/100] step: [17/12] time: 0.360s, mse: 0.023 
Ep

Epoch: [94/100] step: [23/12] time: 0.354s, mse: 0.018 
Epoch: [94/100] step: [24/12] time: 0.355s, mse: 0.018 
Epoch: [95/100] step: [0/12] time: 0.470s, mse: 0.021 
Epoch: [95/100] step: [1/12] time: 0.481s, mse: 0.025 
Epoch: [95/100] step: [2/12] time: 0.434s, mse: 0.018 
Epoch: [95/100] step: [3/12] time: 0.385s, mse: 0.014 
Epoch: [95/100] step: [4/12] time: 0.475s, mse: 0.025 
Epoch: [95/100] step: [5/12] time: 0.608s, mse: 0.025 
Epoch: [95/100] step: [6/12] time: 0.462s, mse: 0.021 
Epoch: [95/100] step: [7/12] time: 0.430s, mse: 0.016 
Epoch: [95/100] step: [8/12] time: 0.366s, mse: 0.019 
Epoch: [95/100] step: [9/12] time: 0.361s, mse: 0.029 
Epoch: [95/100] step: [10/12] time: 0.357s, mse: 0.018 
Epoch: [95/100] step: [11/12] time: 0.357s, mse: 0.016 
Epoch: [95/100] step: [12/12] time: 0.367s, mse: 0.029 
Epoch: [95/100] step: [13/12] time: 0.362s, mse: 0.015 
Epoch: [95/100] step: [14/12] time: 0.504s, mse: 0.023 
Epoch: [95/100] step: [15/12] time: 0.365s, mse: 0.033 
Ep

In [ ]:
G.get_weights(os.path.join(checkpoint_dir, 'g-initial.h5'))

In [ ]:
G.

In [25]:
G.save_weights(os.path.join(checkpoint_dir, 'g-initial.h5'))
session.upload_data(bucket_path="/mipt-course/tendai/srgan-checkpoints-base", path=checkpoint_dir)

[TL] [*] Saving TL weights into checkpoint/g-initial.h5


INFO:tensorlayer:[*] Saving TL weights into checkpoint/g-initial.h5


[TL] [*] Saved


INFO:tensorlayer:[*] Saved


Successfully upload file checkpoint to OBS mipt-course/tendai/srgan-checkpoints-base


In [ ]:
## adversarial learning (G, D)
n_step_epoch = round(n_epoch // batch_size)
for epoch in range(n_epoch):
    for step, (lr_patchs, hr_patchs) in enumerate(train_ds):
        if lr_patchs.shape[0] != batch_size: # if the remaining data in this epoch < batch_size
            break
        step_time = time.time()
        with tf.GradientTape(persistent=True) as tape:
            fake_patchs = G(lr_patchs)
            logits_fake = D(fake_patchs)
            logits_real = D(hr_patchs)
            feature_fake = VGG((fake_patchs+1)/2.) # the pre-trained VGG uses the input range of [0, 1]
            feature_real = VGG((hr_patchs+1)/2.)
            d_loss1 = tl.cost.sigmoid_cross_entropy(logits_real, tf.ones_like(logits_real))
            d_loss2 = tl.cost.sigmoid_cross_entropy(logits_fake, tf.zeros_like(logits_fake))
            d_loss = d_loss1 + d_loss2
            g_gan_loss = 1e-3 * tl.cost.sigmoid_cross_entropy(logits_fake, tf.ones_like(logits_fake))

            mse_loss = tl.cost.mean_squared_error(fake_patchs, hr_patchs, is_mean=True)
            
            vgg_loss = 2e-6 * tl.cost.mean_squared_error(feature_fake, feature_real, is_mean=True)
            g_loss = mse_loss + vgg_loss + g_gan_loss
        grad = tape.gradient(g_loss, G.trainable_weights)
        g_optimizer.apply_gradients(zip(grad, G.trainable_weights))
        grad = tape.gradient(d_loss, D.trainable_weights)
        d_optimizer.apply_gradients(zip(grad, D.trainable_weights))
        
        print("Epoch: [{}/{}] step: [{}/{}] time: {:.3f}s, g_loss(mse:{:.3f}, vgg:{:.3f}, adv:{:.3f}) d_loss: {:.3f}".format(
            epoch, n_epoch_init, step, n_step_epoch, time.time() - step_time, mse_loss, vgg_loss, g_gan_loss, d_loss))

    # update the learning rate
    if epoch != 0 and (epoch % decay_every == 0):
        new_lr_decay = lr_decay**(epoch // decay_every)
        lr_v.assign(lr_init * new_lr_decay)
        log = " ** new learning rate: %f (for GAN)" % (lr_init * new_lr_decay)
        print(log)

    if (epoch != 0) and (epoch % 10 == 0):
        tl.vis.save_images(fake_patchs.numpy(), [2, 4], os.path.join(save_dir, 'train_g_{}.png'.format(epoch)))
        G.save_weights(os.path.join(checkpoint_dir, 'g-{epoch}.h5'.format(epoch=epoch)))
        D.save_weights(os.path.join(checkpoint_dir, 'd-{epoch}.h5'.format(epoch=epoch)))
        session.upload_data(bucket_path="/mipt-course/tendai/srgan-samples-base/samples", path=save_dir)
        session.upload_data(bucket_path="/mipt-course/tendai/srgan-checkpoints-base", path=checkpoint_dir)
        

Epoch: [0/100] step: [0/250] time: 1.551s, g_loss(mse:0.024, vgg:0.045, adv:0.001) d_loss: 1.518
Epoch: [0/100] step: [1/250] time: 1.667s, g_loss(mse:0.033, vgg:0.066, adv:0.001) d_loss: 1.800
Epoch: [0/100] step: [2/250] time: 1.507s, g_loss(mse:0.011, vgg:0.029, adv:0.000) d_loss: 1.688
Epoch: [0/100] step: [3/250] time: 1.501s, g_loss(mse:0.018, vgg:0.057, adv:0.000) d_loss: 1.722
Epoch: [0/100] step: [4/250] time: 1.490s, g_loss(mse:0.025, vgg:0.065, adv:0.001) d_loss: 1.506
Epoch: [0/100] step: [5/250] time: 1.474s, g_loss(mse:0.017, vgg:0.046, adv:0.001) d_loss: 1.664
Epoch: [0/100] step: [6/250] time: 1.490s, g_loss(mse:0.016, vgg:0.038, adv:0.001) d_loss: 1.556
Epoch: [0/100] step: [7/250] time: 1.470s, g_loss(mse:0.027, vgg:0.063, adv:0.001) d_loss: 1.595
Epoch: [0/100] step: [8/250] time: 1.405s, g_loss(mse:0.017, vgg:0.045, adv:0.001) d_loss: 1.594
Epoch: [0/100] step: [9/250] time: 1.407s, g_loss(mse:0.030, vgg:0.057, adv:0.001) d_loss: 1.428
Epoch: [0/100] step: [10/250] 

Epoch: [3/100] step: [9/250] time: 1.451s, g_loss(mse:0.028, vgg:0.055, adv:0.001) d_loss: 1.406
Epoch: [3/100] step: [10/250] time: 1.427s, g_loss(mse:0.023, vgg:0.050, adv:0.001) d_loss: 1.940
Epoch: [3/100] step: [11/250] time: 1.443s, g_loss(mse:0.010, vgg:0.038, adv:0.001) d_loss: 1.622
Epoch: [3/100] step: [12/250] time: 1.433s, g_loss(mse:0.032, vgg:0.046, adv:0.001) d_loss: 1.559
Epoch: [3/100] step: [13/250] time: 1.432s, g_loss(mse:0.016, vgg:0.028, adv:0.001) d_loss: 1.427
Epoch: [3/100] step: [14/250] time: 1.427s, g_loss(mse:0.022, vgg:0.041, adv:0.001) d_loss: 1.435
Epoch: [3/100] step: [15/250] time: 1.417s, g_loss(mse:0.037, vgg:0.045, adv:0.001) d_loss: 1.416
Epoch: [3/100] step: [16/250] time: 1.666s, g_loss(mse:0.014, vgg:0.033, adv:0.001) d_loss: 1.448
Epoch: [3/100] step: [17/250] time: 1.406s, g_loss(mse:0.013, vgg:0.040, adv:0.001) d_loss: 1.466
Epoch: [3/100] step: [18/250] time: 1.405s, g_loss(mse:0.018, vgg:0.067, adv:0.001) d_loss: 1.655
Epoch: [3/100] step: 

Epoch: [6/100] step: [18/250] time: 1.407s, g_loss(mse:0.022, vgg:0.068, adv:0.001) d_loss: 1.519
Epoch: [6/100] step: [19/250] time: 1.412s, g_loss(mse:0.011, vgg:0.044, adv:0.001) d_loss: 1.517
Epoch: [6/100] step: [20/250] time: 1.426s, g_loss(mse:0.009, vgg:0.032, adv:0.001) d_loss: 1.536
Epoch: [6/100] step: [21/250] time: 1.595s, g_loss(mse:0.012, vgg:0.033, adv:0.001) d_loss: 1.474
Epoch: [6/100] step: [22/250] time: 1.415s, g_loss(mse:0.039, vgg:0.070, adv:0.001) d_loss: 1.409
Epoch: [6/100] step: [23/250] time: 1.424s, g_loss(mse:0.010, vgg:0.030, adv:0.001) d_loss: 1.453
Epoch: [6/100] step: [24/250] time: 1.412s, g_loss(mse:0.007, vgg:0.028, adv:0.001) d_loss: 1.565
Epoch: [7/100] step: [0/250] time: 1.517s, g_loss(mse:0.024, vgg:0.047, adv:0.001) d_loss: 1.365
Epoch: [7/100] step: [1/250] time: 1.486s, g_loss(mse:0.020, vgg:0.053, adv:0.001) d_loss: 1.454
Epoch: [7/100] step: [2/250] time: 1.481s, g_loss(mse:0.024, vgg:0.056, adv:0.001) d_loss: 1.530
Epoch: [7/100] step: [3

Epoch: [10/100] step: [2/250] time: 1.534s, g_loss(mse:0.013, vgg:0.031, adv:0.002) d_loss: 1.568
Epoch: [10/100] step: [3/250] time: 1.495s, g_loss(mse:0.012, vgg:0.038, adv:0.001) d_loss: 1.554
Epoch: [10/100] step: [4/250] time: 1.504s, g_loss(mse:0.026, vgg:0.060, adv:0.001) d_loss: 1.874
Epoch: [10/100] step: [5/250] time: 1.478s, g_loss(mse:0.014, vgg:0.041, adv:0.001) d_loss: 1.202
Epoch: [10/100] step: [6/250] time: 1.487s, g_loss(mse:0.014, vgg:0.030, adv:0.001) d_loss: 1.217
Epoch: [10/100] step: [7/250] time: 1.482s, g_loss(mse:0.018, vgg:0.055, adv:0.000) d_loss: 1.321
Epoch: [10/100] step: [8/250] time: 1.424s, g_loss(mse:0.016, vgg:0.040, adv:0.001) d_loss: 1.327
Epoch: [10/100] step: [9/250] time: 1.412s, g_loss(mse:0.029, vgg:0.053, adv:0.001) d_loss: 1.357
Epoch: [10/100] step: [10/250] time: 1.415s, g_loss(mse:0.026, vgg:0.058, adv:0.001) d_loss: 1.301
Epoch: [10/100] step: [11/250] time: 1.591s, g_loss(mse:0.013, vgg:0.027, adv:0.002) d_loss: 1.424
Epoch: [10/100] st

INFO:tensorlayer:[*] Saving TL weights into checkpoint/g-10.h5


...

INFO:tensorlayer:[*] Saving TL weights into checkpoint/d-300.h5


[TL] [*] Saved


INFO:tensorlayer:[*] Saved


Successfully upload file samples to OBS mipt-course/tendai/srgan-samples-base/samples
Successfully upload file checkpoint to OBS mipt-course/tendai/srgan-checkpoints-base
Epoch: [301/100] step: [0/250] time: 1.465s, g_loss(mse:0.018, vgg:0.034, adv:0.000) d_loss: 1.298
Epoch: [301/100] step: [1/250] time: 1.632s, g_loss(mse:0.023, vgg:0.057, adv:0.002) d_loss: 0.123
Epoch: [301/100] step: [2/250] time: 1.473s, g_loss(mse:0.018, vgg:0.036, adv:0.005) d_loss: 0.014
Epoch: [301/100] step: [3/250] time: 1.465s, g_loss(mse:0.015, vgg:0.033, adv:0.003) d_loss: 0.453
Epoch: [301/100] step: [4/250] time: 1.456s, g_loss(mse:0.022, vgg:0.050, adv:0.004) d_loss: 0.031
Epoch: [301/100] step: [5/250] time: 1.468s, g_loss(mse:0.021, vgg:0.037, adv:0.004) d_loss: 0.021
Epoch: [301/100] step: [6/250] time: 1.493s, g_loss(mse:0.027, vgg:0.061, adv:0.005) d_loss: 0.009
Epoch: [301/100] step: [7/250] time: 1.447s, g_loss(mse:0.018, vgg:0.042, adv:0.005) d_loss: 0.030
Epoch: [301/100] step: [8/250] time: 

Epoch: [304/100] step: [6/250] time: 1.468s, g_loss(mse:0.021, vgg:0.045, adv:0.005) d_loss: 0.021
Epoch: [304/100] step: [7/250] time: 1.498s, g_loss(mse:0.026, vgg:0.064, adv:0.005) d_loss: 0.009
Epoch: [304/100] step: [8/250] time: 1.409s, g_loss(mse:0.019, vgg:0.044, adv:0.005) d_loss: 0.283
Epoch: [304/100] step: [9/250] time: 1.423s, g_loss(mse:0.025, vgg:0.050, adv:0.005) d_loss: 0.014
Epoch: [304/100] step: [10/250] time: 1.401s, g_loss(mse:0.021, vgg:0.059, adv:0.006) d_loss: 0.008
Epoch: [304/100] step: [11/250] time: 1.394s, g_loss(mse:0.014, vgg:0.027, adv:0.005) d_loss: 0.650
Epoch: [304/100] step: [12/250] time: 1.402s, g_loss(mse:0.021, vgg:0.032, adv:0.004) d_loss: 0.037
Epoch: [304/100] step: [13/250] time: 1.409s, g_loss(mse:0.013, vgg:0.027, adv:0.004) d_loss: 0.073
Epoch: [304/100] step: [14/250] time: 1.566s, g_loss(mse:0.026, vgg:0.040, adv:0.002) d_loss: 0.117
Epoch: [304/100] step: [15/250] time: 1.401s, g_loss(mse:0.017, vgg:0.036, adv:0.004) d_loss: 0.044
Epoc